## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\Kathy\Git\School 2022\School Challenges\World_Weather_Analysis\Weather_Database/weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,2022-06-23 13:09:53,-46.1927,168.8643,33.48,81,3,3.69,clear sky
1,1,Alekseyevsk,RU,2022-06-23 13:09:54,57.8406,108.3492,55.18,95,93,2.48,light rain
2,2,Hilo,US,2022-06-23 13:06:59,19.7297,-155.0900,71.28,91,100,4.61,light rain
3,3,Dikson,RU,2022-06-23 13:09:54,73.5069,80.5464,43.50,74,100,20.47,overcast clouds
4,4,Saldanha,ZA,2022-06-23 13:09:55,-33.0117,17.9442,55.54,88,100,25.41,moderate rain


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Cabo San Lucas,MX,2022-06-23 13:09:19,22.8909,-109.9124,81.61,82,76,2.86,broken clouds
7,7,Rikitea,PF,2022-06-23 13:09:56,-23.1203,-134.9692,75.22,65,79,15.84,broken clouds
9,9,Kahului,US,2022-06-23 13:09:57,20.8947,-156.4700,75.22,77,100,9.22,overcast clouds
10,10,Sao Miguel Do Araguaia,BR,2022-06-23 13:09:57,-13.2750,-50.1628,85.62,33,0,7.61,clear sky
11,11,Camocim,BR,2022-06-23 13:06:55,-2.9022,-40.8411,82.27,65,23,10.22,few clouds
14,14,Butaritari,KI,2022-06-23 13:09:59,3.0707,172.7902,81.77,76,17,11.27,few clouds
18,18,Tambura,SS,2022-06-23 13:10:01,5.6000,27.4667,89.56,44,100,3.31,overcast clouds
20,20,Karlstad,SE,2022-06-23 13:10:03,59.3793,13.5036,75.76,38,0,10.36,clear sky
21,21,Gurupi,BR,2022-06-23 13:07:30,-11.7292,-49.0686,84.15,37,0,11.92,clear sky
25,25,Mumford,GH,2022-06-23 13:10:06,5.2618,-0.7590,78.46,81,100,8.99,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                274
City                   274
Country                274
Date                   274
Lat                    274
Lng                    274
Max Temp               274
Humidity               274
Cloudiness             274
Wind Speed             274
Current Description    274
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                274
City                   274
Country                274
Date                   274
Lat                    274
Lng                    274
Max Temp               274
Humidity               274
Cloudiness             274
Wind Speed             274
Current Description    274
dtype: int64

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Cabo San Lucas,MX,81.61,broken clouds,22.8909,-109.9124,
7,Rikitea,PF,75.22,broken clouds,-23.1203,-134.9692,
9,Kahului,US,75.22,overcast clouds,20.8947,-156.4700,
10,Sao Miguel Do Araguaia,BR,85.62,clear sky,-13.2750,-50.1628,
11,Camocim,BR,82.27,few clouds,-2.9022,-40.8411,
14,Butaritari,KI,81.77,few clouds,3.0707,172.7902,
18,Tambura,SS,89.56,overcast clouds,5.6000,27.4667,
20,Karlstad,SE,75.76,clear sky,59.3793,13.5036,
21,Gurupi,BR,84.15,clear sky,-11.7292,-49.0686,
25,Mumford,GH,78.46,overcast clouds,5.2618,-0.7590,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df = hotel_df.replace("", nan_value)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Cabo San Lucas,MX,81.61,broken clouds,22.8909,-109.9124,Vista Roof Restaurant
7,Rikitea,PF,75.22,broken clouds,-23.1203,-134.9692,People ThankYou
9,Kahului,US,75.22,overcast clouds,20.8947,-156.4700,Maui Seaside Hotel
10,Sao Miguel Do Araguaia,BR,85.62,clear sky,-13.2750,-50.1628,Hotel Executivo Palace
11,Camocim,BR,82.27,few clouds,-2.9022,-40.8411,Ilha Praia Hotel
...,...,...,...,...,...,...,...
671,The Valley,AI,85.86,scattered clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
672,Santa Isabel,BR,76.30,clear sky,-23.3156,-46.2214,Chácara Sombra do Altíssimo
676,Camp Verde,US,81.27,clear sky,34.5636,-111.8543,Days Inn by Wyndham Camp Verde Arizona
677,Allapalli,IN,80.29,moderate rain,19.4167,80.0667,"Hotel IK Residency, GUEST HOUSE AC NON AC"


In [36]:
# 8a. Create the output File (CSV)
output_file = "weatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
# Configure gmaps API key
gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))